In [1]:
import numpy as np
import pandas as pd

from PIL import Image

In [2]:
fisier_txt_train = pd.read_csv("./data/train.txt", sep=',')

fisier_txt_validation = pd.read_csv("./data/validation.txt", sep=',')

fisier_txt_test = pd.read_csv("./data/test.txt", sep=',')

In [3]:
print(fisier_txt_train.id[4:8])
print("----------------------------")
print(fisier_txt_train.id[5:7])

4    ERUZFIFntfyer1B.png
5    TcmRDSNgsQ0Pbjs.png
6    3DbRytghuEd5UFD.png
7    XsF3J1aihqTJmlF.png
Name: id, dtype: object
----------------------------
5    ir4n790lQWRvUoz.png
6    x6uhp7UV7Gv3Xez.png
Name: id, dtype: object


In [50]:
def return_pixel_rgb_values(img_name, path = "../data/train_validation/") :
    return np.array(Image.open(path + img_name).getdata()).reshape((16, 16, 3))

In [29]:
image_name_test = fisier_txt_train.id[3]

image = return_pixel_rgb_values(image_name_test)

# print(image[0])
print(image.shape)

(16, 16, 3)


In [31]:
train_images = np.array([return_pixel_rgb_values(x) for x in train_data.id])

validation_images = np.array([return_pixel_rgb_values(x) for x in validation_data.id]).reshape((1173, 768))/255

In [33]:
train_rotate_90 = np.rot90(train_images, k=1, axes =(0, 1)).reshape((8000,768))/255
train_images = train_images.reshape((8000, 768))/255

In [55]:
test_images = np.array([return_pixel_rgb_values(x, "./data/test/") for x in test_data.id]).reshape((2819, 768))/255

In [56]:
train_mean_image = np.mean(train_images, axis=0)

train_90_mean = np.mean(train_rotate_90, axis=0)

validation_mean_image = np.mean(validation_images, axis=0)

test_mean_image = np.mean(test_images, axis=0)

In [57]:
train_images_standardized = train_images - train_mean_image / np.std(train_images)

train_90_standardized = train_rotate_90 - train_90_mean / np.std(train_rotate_90)

validation_images_standardized = validation_images - validation_mean_image/np.std(validation_images)

test_images_standardized = test_images - test_mean_image / np.std(test_images)

In [41]:
def classifica_imaginea(train_images, train_labels, image_pixels, number_of_neighbours = 1) :
    # luam in calcul metrica l2
    distante = np.sqrt(np.sum(np.power((train_images - image_pixels), 2), axis=1))

    sort_indexes = np.argsort(distante)

    sort_indexes = sort_indexes[ :number_of_neighbours]

    labels = train_labels[sort_indexes]


    return np.argmax(np.bincount(labels))

In [37]:
images_perturbed = []
for img in train_images_standardized :
    images_perturbed.append(img + np.random.normal(loc=0.0,scale=0.01, size=(768,)))
images_perturbed = np.array(images_perturbed)

In [43]:
np.concatenate((train_images_standardized, train_rotate_90), axis=0)[0]

array([-0.63198344, -0.52780605, -0.43828016, -0.63135429, -0.59007048,
       -0.4910649 , -0.6254513 , -0.57349237, -0.47927542, -0.66150679,
       -0.59629774, -0.52848484, -0.61952495, -0.56346306, -0.50864333,
       -0.61995092, -0.5442124 , -0.50826409, -0.60124576, -0.52784865,
       -0.49416205, -0.60638476, -0.54468508, -0.51473182, -0.62119444,
       -0.55115281, -0.51039252, -0.63502167, -0.56885078, -0.51765995,
       -0.65326927, -0.5901914 , -0.53182107, -0.6482869 , -0.57452979,
       -0.5182934 , -0.6289167 , -0.54165939, -0.50570833, -0.61407266,
       -0.51470433, -0.48367243, -0.60386885, -0.51744147, -0.47678836,
       -0.57907108, -0.5295415 , -0.45074844, -0.65446305, -0.5587349 ,
       -0.4614689 , -0.57241098, -0.52675902, -0.4351308 , -0.59119309,
       -0.52853431, -0.44781618, -0.56595974, -0.50008015, -0.44371597,
       -0.53468325, -0.48096277, -0.43653372, -0.55183297, -0.47914488,
       -0.45587095, -0.52652405, -0.45235609, -0.43097013, -0.52

In [63]:
a = np.concatenate((train_images_standardized, train_90_standardized), axis=0)
a_labels = np.concatenate((np.array(train_data.label),np.array(train_data.label)), axis=0)

(16000,)

In [46]:
if True :
    num = 12
    validation_knn_labels = []

    for image_pixels in validation_images_standardized:
        validation_knn_labels.append(classifica_imaginea(np.concatenate((train_images_standardized, train_90_standardized), axis=0), np.array(train_data.label), image_pixels, num))

    validation_knn_labels = np.array(validation_knn_labels)


In [48]:
 print(f"Valoarea de prezicere pentru \tnr vecini = \t{num} \teste = \t{(np.array(validation_data.label) == validation_knn_labels).mean()}")

Valoarea de prezicere pentru 	nr vecini = 	12 	este = 	0.4381926683716965


In [15]:
for num in [10, 15, 20, 25] :
    validation_knn_labels = []

    for image_pixels in validation_images_standardized:
        validation_knn_labels.append(classifica_imaginea(train_images_standardized, np.array(train_data.label), image_pixels, num))

    validation_knn_labels = np.array(validation_knn_labels)

    print(f"Valoarea de prezicere pentru \tnr vecini = \t{num} \teste = \t{(np.array(validation_data.label) == validation_knn_labels).mean()}")

Valoarea de prezicere pentru 	nr vecini = 	10 	este = 	0.4356351236146633
Valoarea de prezicere pentru 	nr vecini = 	15 	este = 	0.4339300937766411
Valoarea de prezicere pentru 	nr vecini = 	20 	este = 	0.4322250639386189
Valoarea de prezicere pentru 	nr vecini = 	25 	este = 	0.4339300937766411


In [18]:
for num in [12, 23] :
    validation_knn_labels = []

    for image_pixels in validation_images_standardized:
        validation_knn_labels.append(classifica_imaginea(train_images_standardized, np.array(train_data.label), image_pixels, num))

    validation_knn_labels = np.array(validation_knn_labels)

    print(f"Valoarea de prezicere pentru \tnr vecini = \t{num} \teste = \t{(np.array(validation_data.label) == validation_knn_labels).mean()}")

Valoarea de prezicere pentru 	nr vecini = 	12 	este = 	0.4381926683716965
Valoarea de prezicere pentru 	nr vecini = 	23 	este = 	0.4322250639386189


In [23]:
for num in [11, 13] :
    validation_knn_labels = []

    for image_pixels in validation_images_standardized:
        validation_knn_labels.append(classifica_imaginea(train_images_standardized, np.array(train_data.label), image_pixels, num))

    validation_knn_labels = np.array(validation_knn_labels)

    print(f"Valoarea de prezicere pentru \tnr vecini = \t{num} \teste = \t{(np.array(validation_data.label) == validation_knn_labels).mean()}")

Valoarea de prezicere pentru 	nr vecini = 	11 	este = 	0.4356351236146633
Valoarea de prezicere pentru 	nr vecini = 	13 	este = 	0.43478260869565216


In [16]:
np.unique(np.array(train_data.label), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6], dtype=int64),
 array([1103, 1134, 1162, 1155, 1164, 1145, 1137], dtype=int64))

In [17]:
np.unique(np.array(validation_data.label), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6], dtype=int64),
 array([216, 201, 142, 150, 143, 145, 176], dtype=int64))

In [65]:
if True :
    num = 12
    validation_knn_labels = []

    for image_pixels in test_images_standardized:

        label = classifica_imaginea(a, a_labels, image_pixels, num)
        validation_knn_labels.append(label)

In [68]:
f = open("knn_first_try_12.txt","w")
f.write("id,label\n")
i = 0
for nume_img in test_data.id :
    f.write(f"{nume_img},{validation_knn_labels[i]}\n")
    i += 1